In [ ]:
import pandas as pd
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
import streamlit as st
import random
from pyngrok import ngrok
import spotipy

In [2]:
# # # # Install the necessary packages
!pip install streamlit
!pip install pyngrok==4.1.1
!pip install transformers
!pip install -U sentence-transformers
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.2 MB/s eta 0:00:00
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 7.0.0
    Uninstalling importlib-metadata-7.0.0:
      Successfully uninstalled importlib-metadata-7.0.0
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=52165e303024dce08ee089045777dc7ad28651ff44c40c3b0a3442bdc582e71f
  Stored in directory: /root/.cache/pip/wheels/4c/7c/4c/632fba2ea8e88d8890102eb07bc922e1ca8fa14db5902c91a8
Successfully built pyngrok
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.0 MB/s et

In [ ]:
# import pandas as pd
# # List of file names
# files = [
#     '00_10_10000rows.csv',
#     '10_20_10000rows.csv',
#     '20_30_10000rows.csv',
#     '30_40_10000rows.csv',
#     '40_50_10000rows.csv',
#     '50_60_10000rows.csv',
#     '60_70_10000rows.csv',
#     '70_80_10000rows.csv',
#     '80_90_10000rows.csv',
#     '90_100_10000rows.csv',
#     '100_110_10000rows.csv',
#     '110_120_10000rows.csv',
#     '120_130_10000rows.csv',
#     '130_140_10000rows.csv',
#     '140_all_10000rows.csv'
# ]

# # Read each file into a DataFrame and store in a list
# dfs = [pd.read_csv(file) for file in files]

# # Concatenate all DataFrames consecutively
# full_df = pd.concat(dfs, ignore_index=True)

# # Save the concatenated DataFrame to a new CSV file
# full_df.to_csv('music_mental_health.csv', index=False)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def analyze_user_input(user_input):
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    model = TFRobertaForSequenceClassification.from_pretrained('arpanghoshal/EmoRoBERTa')
    encoded_input = tokenizer(user_input, return_tensors="tf", truncation=True, padding=True, max_length=512)
    outputs = model(encoded_input)
    scores = tf.nn.softmax(outputs.logits, axis=-1).numpy()[0]
    predicted_class_idx = tf.argmax(outputs.logits, axis=-1).numpy()[0]
    sentiment_label = model.config.id2label[predicted_class_idx]
    sentiment_score = scores[predicted_class_idx]
    return sentiment_label, sentiment_score

In [ ]:
from sentence_transformers import SentenceTransformer, util

model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
def match_songs_with_sentiment(user_sentiment_label, user_sentiment_score,inputVector, score_range=0.1):
    # Load the dataset of songs with their sentiment and score
    songs_df = pd.read_csv('/content/music_mental_health.csv')

    # Filter songs with the same sentiment label
    matched_songs = songs_df[songs_df['sentiment'] == user_sentiment_label]

    # Calculate the score range
    score_min = max(0, user_sentiment_score - score_range / 2)
    score_max = min(1, user_sentiment_score + score_range / 2)

    # Further filter songs whose scores fall within the specified range
    matched_songs = matched_songs[(matched_songs['score'] >= score_min) & (matched_songs['score'] <= score_max)]

    # Shuffle the matched songs to get a random order
    matched_songs = matched_songs.sample(frac=1).reset_index(drop=True)

    matched_songs['similarity'] = matched_songs['seq'].apply(lambda x: util.pytorch_cos_sim(model.encode(x), inputVector))

    top_5 = matched_songs['similarity'].sort_values(ascending=False).head(5)

    # Sort the songs by how close their score is to the user's sentiment score
    # matched_songs['score_diff'] = abs(matched_songs['score'] - user_sentiment_score)
    # matched_songs = matched_songs.sort_values(by='score_diff')

    # Select the top five songs and return
    return matched_songs.loc[top_5.index, ['song','artist','seq','similarity','score']]

# New Section

In [ ]:
def suggest_songs_for_user():
    user_input = input("How was your day? Tell me about it: ")
    inputVector = model.encode(user_input)
    user_sentiment_label, user_sentiment_score = analyze_user_input(user_input)
    print(f"Your sentiment label is '{user_sentiment_label}' with a score of {user_sentiment_score:.2f}")
    suggested_songs = match_songs_with_sentiment(user_sentiment_label, user_sentiment_score, inputVector, 0.00625)
    return suggested_songs

In [ ]:
suggested_songs = suggest_songs_for_user()
print("Based on your day, you might like these songs:")
print(suggested_songs[['song','artist','seq','similarity','score']])

How was your day? Tell me about it: I met a girl today and I fell in love


All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at arpanghoshal/EmoRoBERTa.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


Your sentiment label is 'love' with a score of 0.99
Based on your day, you might like these songs:
                          song         artist  \
907   Hallelujah I Love Her So  Guy Sebastian   
1200      The Love (Interlude)        Tove Lo   
582     Girl of My Best Friend      Bobby Vee   
2264          In Love with You    Freestylers   
169        Bonaparte's Retreat  Willie Nelson   

                                                    seq          similarity  \
907   let me tell you 'bout a girl i know she is my ...  [[tensor(0.4560)]]   
1200  and then you freak out 'cause suddenly you lov...  [[tensor(0.4487)]]   
582   the way she walks, the way she talks how long ...  [[tensor(0.4439)]]   
2264  i'm in love with you i know i saw you yesterda...  [[tensor(0.4403)]]   
169   met a girl that i loved in a town way down in ...  [[tensor(0.4325)]]   

         score  
907   0.986515  
1200  0.986873  
582   0.988238  
2264  0.987667  
169   0.987824  


In [ ]:
!pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 9.7 MB/s eta 0:00:00


In [ ]:
# # Function definitions go here
# # ... (insert the analyze_user_input and match_songs_with_sentiment functions from before) ...

# # Initialize the tokenizer and model outside of the function to speed up repeated calls
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = TFRobertaForSequenceClassification.from_pretrained('arpanghoshal/EmoRoBERTa')

# # Streamlit app layout
# st.title('Music Suggestion Based on Your Mood')

# # User input text area
# user_input = st.text_area("How was your day? Tell me about it:", height=150)

# if user_input:
#     # Run sentiment analysis on the user input
#     sentiment_label, sentiment_score = analyze_user_input(user_input)
#     st.write(f"Sentiment: {sentiment_label}, Score: {sentiment_score:.2f}")

#     # Suggest songs
#     songs_df = pd.read_csv('music_mental_health.csv')
#     suggested_songs = match_songs_with_sentiment(sentiment_label, sentiment_score)
#     st.write("Based on your mood, you might like these songs:")
#     st.dataframe(suggested_songs['song', 'artist', 'sentiment','score']])


In [ ]:
# # Write the Streamlit app to a file
# %%writefile app.py
# import streamlit as st
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
# from sentence_transformers import SentenceTransformer, util
# import spotipy
# from spotipy.oauth2 import SpotifyClientCredentials

# sim_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# # Function to analyze sentiment of the user's input
# def analyze_user_input(user_input, tokenizer, model):
#     encoded_input = tokenizer(user_input, return_tensors="tf", truncation=True, padding=True, max_length=512)
#     outputs = model(encoded_input)
#     scores = tf.nn.softmax(outputs.logits, axis=-1).numpy()[0]
#     predicted_class_idx = tf.argmax(outputs.logits, axis=-1).numpy()[0]
#     sentiment_label = model.config.id2label[predicted_class_idx]
#     sentiment_score = scores[predicted_class_idx]
#     return sentiment_label, sentiment_score

# # Function to match songs from the dataset with the user's sentiment
# def match_songs_with_sentiment(user_sentiment_label, user_sentiment_score,inputVector, score_range,songs_df):

#     # Filter songs with the same sentiment label
#     matched_songs = songs_df[songs_df['sentiment'] == user_sentiment_label]

#     # Calculate the score range
#     score_min = max(0, user_sentiment_score - score_range)
#     score_max = min(1, user_sentiment_score + score_range)

#     # Further filter songs whose scores fall within the specified range
#     matched_songs = matched_songs[(matched_songs['score'] >= score_min) & (matched_songs['score'] <= score_max)]

#     # Shuffle the matched songs to get a random order
#     matched_songs = matched_songs.sample(frac=1).reset_index(drop=True)

#     matched_songs['similarity'] = matched_songs['seq'].apply(lambda x: util.pytorch_cos_sim(sim_model.encode(x), inputVector))

#     top_5 = matched_songs['similarity'].sort_values(ascending=False).head(5)

#     # Sort the songs by how close their score is to the user's sentiment score
#     # matched_songs['score_diff'] = abs(matched_songs['score'] - user_sentiment_score)
#     # matched_songs = matched_songs.sort_values(by='score_diff')

#     # Select the top five songs and return
#     return matched_songs.loc[top_5.index, ['song','artist','seq','similarity','sentiment','score']]

# client_id = 'c34955a27b6447e3a1b92305d04bbbea'
# client_secret = '1d197925c0654b5da80bd3cfa1f5afdd'

# # Initialize the Spotify API client
# client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
# sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# # Function to get the track ID for a song
# def get_track_id(song_name):
#     # Search for the track ID using the song name
#     results = sp.search(q=song_name, type='track', limit=1)
#     if results['tracks']['items']:
#         track_id = results['tracks']['items'][0]['id']
#         return track_id
#     else:
#         print(f"No results found for {song_name}")
#         return None

# # Function to get the preview URL for a song
# def get_track_preview_url(track_id):
#     # Get the 30-second preview URL for the track
#     track_info = sp.track(track_id)
#     preview_url = track_info['preview_url']
#     return preview_url

# # Initialize the tokenizer and model outside of the functions to speed up repeated calls
# tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
# model = TFRobertaForSequenceClassification.from_pretrained('arpanghoshal/EmoRoBERTa')

# # Streamlit app layout
# st.title('Music Suggestion Based on Your Mood')

# # User input text area
# user_input = st.text_area("How was your day? Tell me about it:", key="123", height=150)
# submit_button = st.button("Submit")

# if submit_button and len(user_input.split())>5:
#     pass
# elif submit_button and not len(user_input.split())>5:
#     st.warning("Please provide a longer response with 5 words or more.")
#     st.rerun()

# #user_input = st.text_area("How was your day? Tell me about it:", height=150)
# inputVector = sim_model.encode(user_input)

# if submit_button:
#     # Run sentiment analysis on the user input
#     sentiment_label, sentiment_score = analyze_user_input(user_input, tokenizer, model)
#     st.write(f"Sentiment: {sentiment_label}, Score: {sentiment_score:.2f}")

#     st.text("Training model... Please wait.")

#     # Load songs dataframe
#     songs_df = pd.read_csv('/content/drive/MyDrive/MusicMentalHealth/music_mental_health.csv')

#     # Suggest songs
#     suggested_songs = match_songs_with_sentiment(sentiment_label, sentiment_score,inputVector,0.00625, songs_df)

#     suggested_songs['similarity'] = suggested_songs['similarity'].apply(lambda x: x.numpy()[0][0])

#     st.write("Based on your mood, you might like these songs:")
#     for index, row in suggested_songs.iterrows():
#       song = row['song']
#       artist = row['artist']
#       track_id = get_track_id(song)
#       if track_id.strip():
#         preview_url = get_track_preview_url(track_id)
#         st.write(f"{song} by {artist}")
#         st.write(f"Lyrics: {row['seq']}")
#         if preview_url:
#           st.audio(preview_url)
#         else:
#           st.write("No Preview Available")
#     st.dataframe(suggested_songs[['song','artist','seq','similarity','sentiment','score']])


Overwriting app.py


In [ ]:
#Above .py file is commented out cause I wanted to keep the original file in case you want to refer back
#Below .py file was edited for the front end part

In [ ]:
'''
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sim_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Function to analyze sentiment of the user's input
def analyze_user_input(user_input, tokenizer, model):
    encoded_input = tokenizer(user_input, return_tensors="tf", truncation=True, padding=True, max_length=512)
    outputs = model(encoded_input)
    scores = tf.nn.softmax(outputs.logits, axis=-1).numpy()[0]
    predicted_class_idx = tf.argmax(outputs.logits, axis=-1).numpy()[0]
    sentiment_label = model.config.id2label[predicted_class_idx]
    sentiment_score = scores[predicted_class_idx]
    return sentiment_label, sentiment_score

# Function to match songs from the dataset with the user's sentiment
def match_songs_with_sentiment(user_sentiment_label, user_sentiment_score,inputVector, score_range,songs_df):

    # Filter songs with the same sentiment label
    matched_songs = songs_df[songs_df['sentiment'] == user_sentiment_label]

    # Calculate the score range
    score_min = max(0, user_sentiment_score - score_range)
    score_max = min(1, user_sentiment_score + score_range)

    # Further filter songs whose scores fall within the specified range
    matched_songs = matched_songs[(matched_songs['score'] >= score_min) & (matched_songs['score'] <= score_max)]

    # Shuffle the matched songs to get a random order
    matched_songs = matched_songs.sample(frac=1).reset_index(drop=True)

    matched_songs['similarity'] = matched_songs['seq'].apply(lambda x: util.pytorch_cos_sim(sim_model.encode(x), inputVector))

    top_5 = matched_songs['similarity'].sort_values(ascending=False).head(5)

    # Sort the songs by how close their score is to the user's sentiment score
    # matched_songs['score_diff'] = abs(matched_songs['score'] - user_sentiment_score)
    # matched_songs = matched_songs.sort_values(by='score_diff')

    # Select the top five songs and return
    return matched_songs.loc[top_5.index, ['song','artist','seq','similarity','sentiment','score']]

client_id = 'c34955a27b6447e3a1b92305d04bbbea'
client_secret = '1d197925c0654b5da80bd3cfa1f5afdd'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_id(song_name):
    # Search for the track ID using the song name
    results = sp.search(q=song_name, type='track', limit=1)
    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
        return track_id
    else:
        print(f"No results found for {song_name}")
        return None

def get_track_preview_url(track_id):
    # Get the 30-second preview URL for the track
    track_info = sp.track(track_id)
    preview_url = track_info['preview_url']
    return preview_url

# Initialize the tokenizer and model outside of the functions to speed up repeated calls
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('arpanghoshal/EmoRoBERTa')

# Streamlit app layout
st.set_page_config(page_title="Music Mood Matcher", layout="wide")  # New: Setting page title and layout

# Custom gradient background using CSS
st.markdown("""
    <style>
    .stApp {
        background: linear-gradient(135deg, #6e8efb, #a777e3);  # Example gradient colors
        background-size: cover;
    }
    </style>
    """, unsafe_allow_html=True)

st.title('Music Suggestion Based on Your Mood')  # Existing Title

# New: Introduction Section
with st.container():
    st.markdown("""
        <style>
        .intro {
            font-size:18px;
        }
        </style>
        <div class='intro'>
        Welcome to the Music Mood Matcher! Share how your day went, and let's find the perfect songs to match your mood.
        Just type in your thoughts, and we'll do the rest.
        </div>
        """, unsafe_allow_html=True)

# User input text area
with st.container():
    user_input = st.text_area("How was your day? Tell me about it:", key="123", height=150, max_chars=500)
    submit_button = st.button("Generate music")

# Processing and Displaying Results
if submit_button and len(user_input.split()) > 5:
    # New: Define inputVector here
    inputVector = sim_model.encode(user_input)

    # Run sentiment analysis on the user input
    sentiment_label, sentiment_score = analyze_user_input(user_input, tokenizer, model)
    st.write(f"Sentiment: {sentiment_label}, Score: {sentiment_score:.2f}")

    # Load songs dataframe
    songs_df = pd.read_csv('/content/drive/MyDrive/MusicMentalHealth/music_mental_health.csv')

    # Suggest songs
    suggested_songs = match_songs_with_sentiment(sentiment_label, sentiment_score, inputVector, 0.00625, songs_df)
    suggested_songs['similarity'] = suggested_songs['similarity'].apply(lambda x: x.numpy()[0][0])

    # Styling for the suggested songs display
    with st.container():
        st.markdown("<div class='song-list'>", unsafe_allow_html=True)
        st.write("Based on your mood, you might like these songs:")
        for index, row in suggested_songs.iterrows():
          song = row['song']
          artist = row['artist']
          track_id = get_track_id(song)
          if track_id.strip():
            preview_url = get_track_preview_url(track_id)
            st.write(f"Similarity: {row['label']}")
            st.write(f"{song} by {artist}")
            st.write(f"Lyrics: {row['seq']}")
            if preview_url:
              st.audio(preview_url)
            else:
              st.write("No Preview Available")
        st.markdown("</div>", unsafe_allow_html=True)
        st.dataframe(suggested_songs[['song','artist','seq','similarity','sentiment','score']])
elif submit_button and not len(user_input.split()) > 5:
    st.warning("Please provide a longer response with 5 words or more.")
    st.rerun()
'''

Writing app.py


In [1]:
%%writefile app.py
#FARHANA
import streamlit as st
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sim_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Function to analyze sentiment of the user's input
def analyze_user_input(user_input, tokenizer, model):
    encoded_input = tokenizer(user_input, return_tensors="tf", truncation=True, padding=True, max_length=512)
    outputs = model(encoded_input)
    scores = tf.nn.softmax(outputs.logits, axis=-1).numpy()[0]
    predicted_class_idx = tf.argmax(outputs.logits, axis=-1).numpy()[0]
    sentiment_label = model.config.id2label[predicted_class_idx]
    sentiment_score = scores[predicted_class_idx]
    return sentiment_label, sentiment_score

# Function to match songs from the dataset with the user's sentiment
def match_songs_with_sentiment(user_sentiment_label, user_sentiment_score,inputVector, score_range,songs_df):

    # Filter songs with the same sentiment label
    matched_songs = songs_df[songs_df['sentiment'] == user_sentiment_label]

    # Calculate the score range
    score_min = max(0, user_sentiment_score - score_range)
    score_max = min(1, user_sentiment_score + score_range)

    # Further filter songs whose scores fall within the specified range
    matched_songs = matched_songs[(matched_songs['score'] >= score_min) & (matched_songs['score'] <= score_max)]

    # Shuffle the matched songs to get a random order
    matched_songs = matched_songs.sample(frac=1).reset_index(drop=True)

    matched_songs['similarity'] = matched_songs['seq'].apply(lambda x: util.pytorch_cos_sim(sim_model.encode(x), inputVector))

    top_5 = matched_songs['similarity'].sort_values(ascending=False).head(5)

    # Sort the songs by how close their score is to the user's sentiment score
    # matched_songs['score_diff'] = abs(matched_songs['score'] - user_sentiment_score)
    # matched_songs = matched_songs.sort_values(by='score_diff')

    # Select the top five songs and return
    return matched_songs.loc[top_5.index, ['song','artist','seq','similarity','sentiment','score']]

client_id = 'c34955a27b6447e3a1b92305d04bbbea'
client_secret = '1d197925c0654b5da80bd3cfa1f5afdd'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_id(song_name):
    # Search for the track ID using the song name
    results = sp.search(q=song_name, type='track', limit=1)
    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
        return track_id
    else:
        print(f"No results found for {song_name}")
        return None

def get_track_preview_url(track_id):
    # Get the 30-second preview URL for the track
    track_info = sp.track(track_id)
    preview_url = track_info['preview_url']
    return preview_url

# Initialize the tokenizer and model outside of the functions to speed up repeated calls
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('arpanghoshal/EmoRoBERTa')

# Streamlit app layout
st.set_page_config(page_title="MODUS MUSIC", layout="wide")  # New: Setting page title and layout
#st.markdown("<h1 style='text-align: center; font-weight: bold;'>MODUS MUSIC</h1>", unsafe_allow_html=True)
image_path = '/content/drive/MyDrive/Colab Notebooks/MODUSMUSIC.png'  # Replace with the actual path to your image

st.image(image_path, use_column_width=False, width=250)  # Adjust the width as needed
# Custom gradient background using CSS
st.markdown("""
    <style>
    .stApp {
        background: rgb(0,0,0);
        background-size: cover;
    }
    </style>
    """, unsafe_allow_html=True)

st.title('Music Suggestion Based on Your Feeling')  # Existing Title

# New: Introduction Section
with st.container():
    st.markdown("""
        <style>
        .intro {
            font-size:18px;
        }
        </style>
        <div class='intro'>
        Welcome to Modus Music! Share your vibe, and let's find the perfect songs to match your mood.
        Just type in your thoughts, and we'll do the rest.
        </div>
        """, unsafe_allow_html=True)

# User input text area
with st.container():
    user_input = st.text_area("What's your vibe? Tell me about it:", key="123", height=150, max_chars=500)
m = st.markdown("""
<style>
div.stButton > button:first-child {
    background-color: rgb(204, 49, 49);

}
</style>""", unsafe_allow_html=True)
# Use the custom style for the button
submit_button = st.button("Generate music")


# Processing and Displaying Results
if submit_button and len(user_input.split()) > 5:
    # New: Define inputVector here
    inputVector = sim_model.encode(user_input)

    # Run sentiment analysis on the user input
    sentiment_label, sentiment_score = analyze_user_input(user_input, tokenizer, model)
    st.write(f"Sentiment: {sentiment_label}, Score: {sentiment_score:.2f}")

    # Load songs dataframe
    songs_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/music_mental_health.csv')

    # Suggest songs
    suggested_songs = match_songs_with_sentiment(sentiment_label, sentiment_score, inputVector, 0.00625, songs_df)
    suggested_songs['similarity'] = suggested_songs['similarity'].apply(lambda x: x.numpy()[0][0])

    # Styling for the suggested songs display
    with st.container():
        st.markdown("<div class='song-list'>", unsafe_allow_html=True)
        st.write("Based on your mood, you might like these songs:")
        for index, row in suggested_songs.iterrows():
          song = row['song']
          artist = row['artist']
          track_id = get_track_id(song)
          if track_id.strip():
            preview_url = get_track_preview_url(track_id)
            #st.write(f"Similarity: {row['similarity']}")
            st.write(f"{song} by {artist}")
            with st.expander(f"Show Lyrics for {song} by {artist}", expanded=False):
                st.write(f"Lyrics: {row['seq']}")

            if preview_url:
              st.audio(preview_url)
            else:
              st.write("No Preview Available")
        st.markdown("</div>", unsafe_allow_html=True)
        st.dataframe(suggested_songs[['song','artist','seq','similarity','sentiment','score']])
elif submit_button and not len(user_input.split()) > 5:
    st.warning("Please provide a longer response with 5 words or more.")
    st.rerun()


Writing app.py


In [ ]:
%%writefile app.py
import streamlit as st
import numpy as np
import pandas as pd
import tensorflow as tf
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sim_model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

# Function to analyze sentiment of the user's input
def analyze_user_input(user_input, tokenizer, model):
    encoded_input = tokenizer(user_input, return_tensors="tf", truncation=True, padding=True, max_length=512)
    outputs = model(encoded_input)
    scores = tf.nn.softmax(outputs.logits, axis=-1).numpy()[0]
    predicted_class_idx = tf.argmax(outputs.logits, axis=-1).numpy()[0]
    sentiment_label = model.config.id2label[predicted_class_idx]
    sentiment_score = scores[predicted_class_idx]
    return sentiment_label, sentiment_score

# Function to match songs from the dataset with the user's sentiment
def match_songs_with_sentiment(user_sentiment_label, user_sentiment_score,inputVector, score_range,songs_df):

    # Filter songs with the same sentiment label
    matched_songs = songs_df[songs_df['sentiment'] == user_sentiment_label]

    # Calculate the score range
    score_min = max(0, user_sentiment_score - score_range)
    score_max = min(1, user_sentiment_score + score_range)

    # Further filter songs whose scores fall within the specified range
    matched_songs = matched_songs[(matched_songs['score'] >= score_min) & (matched_songs['score'] <= score_max)]

    # Shuffle the matched songs to get a random order
    matched_songs = matched_songs.sample(frac=1).reset_index(drop=True)

    matched_songs['similarity'] = matched_songs['seq'].apply(lambda x: util.pytorch_cos_sim(sim_model.encode(x), inputVector))

    top_5 = matched_songs['similarity'].sort_values(ascending=False).head(5)

    # Sort the songs by how close their score is to the user's sentiment score
    # matched_songs['score_diff'] = abs(matched_songs['score'] - user_sentiment_score)
    # matched_songs = matched_songs.sort_values(by='score_diff')

    # Select the top five songs and return
    return matched_songs.loc[top_5.index, ['song','artist','seq','similarity','sentiment','score']]

client_id = 'c34955a27b6447e3a1b92305d04bbbea'
client_secret = '1d197925c0654b5da80bd3cfa1f5afdd'

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_track_id(song_name):
    # Search for the track ID using the song name
    results = sp.search(q=song_name, type='track', limit=1)
    if results['tracks']['items']:
        track_id = results['tracks']['items'][0]['id']
        return track_id
    else:
        print(f"No results found for {song_name}")
        return None

def get_track_preview_url(track_id):
    # Get the 30-second preview URL for the track
    track_info = sp.track(track_id)
    preview_url = track_info['preview_url']
    return preview_url

# Initialize the tokenizer and model outside of the functions to speed up repeated calls
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = TFRobertaForSequenceClassification.from_pretrained('arpanghoshal/EmoRoBERTa')

# Streamlit app layout
st.set_page_config(page_title="MODUS MUSIC", layout="wide")  # New: Setting page title and layout
#st.markdown("<h1 style='text-align: center; font-weight: bold;'>MODUS MUSIC</h1>", unsafe_allow_html=True)
image_path = '/content/MODUSMUSIC.png'  # Replace with the actual path to your image

st.image(image_path, use_column_width=False, width=250)  # Adjust the width as needed
# Custom gradient background using CSS
st.markdown("""
    <style>
    .stApp {
        background: rgb(0,0,0);
        background-size: cover;
    }
    </style>
    """, unsafe_allow_html=True)

st.title('Music Suggestion Based on Your Feelings')  # Existing Title

# New: Introduction Section
with st.container():
    st.markdown("""
        <style>
        .intro {
            font-size:18px;
        }
        </style>
        <div class='intro'>
        Welcome to Modus Music! Share your vibe, and let's find the perfect songs to match your mood.
        Just type in your thoughts, and we'll do the rest.
        </div>
        """, unsafe_allow_html=True)

# User input text area
with st.container():
    user_input = st.text_area("Whats your vibe? Tell me about it:", key="123", height=150, max_chars=500)
m = st.markdown("""
<style>
div.stButton > button:first-child {
    background-color: rgb(204, 49, 49);

}
</style>""", unsafe_allow_html=True)
# Use the custom style for the button
submit_button = st.button("Generate music")


# Processing and Displaying Results
if submit_button and len(user_input.split()) > 5:
    # New: Define inputVector here
    inputVector = sim_model.encode(user_input)

    # Run sentiment analysis on the user input
    sentiment_label, sentiment_score = analyze_user_input(user_input, tokenizer, model)
    st.write(f"Sentiment: {sentiment_label}, Score: {sentiment_score:.2f}")

    # Load songs dataframe
    songs_df = pd.read_csv('/content/music_mental_health.csv')

    # Suggest songs
    suggested_songs = match_songs_with_sentiment(sentiment_label, sentiment_score, inputVector, 0.00625, songs_df)
    suggested_songs['similarity'] = suggested_songs['similarity'].apply(lambda x: x.numpy()[0][0])

    # Styling for the suggested songs display
    with st.container():
        st.markdown("<div class='song-list'>", unsafe_allow_html=True)
        st.write("Based on your mood, you might like these songs:")
        for index, row in suggested_songs.iterrows():
          song = row['song']
          artist = row['artist']
          track_id = get_track_id(song)
          if track_id.strip():
            preview_url = get_track_preview_url(track_id)
            #st.write(f"Similarity: {row['label']}")
            st.write(f"{song} by {artist}")
            with st.expander(f"Show Lyrics for {song} by {artist}", expanded=False):
                st.write(f"Lyrics: {row['seq']}")

            if preview_url:
              st.audio(preview_url)
            else:
              st.write("No Preview Available")
        st.markdown("</div>", unsafe_allow_html=True)
        st.dataframe(suggested_songs[['song','artist','seq','similarity','sentiment','score']])
elif submit_button and not len(user_input.split()) > 5:
    st.warning("Please provide a longer response with 5 words or more.")
    st.rerun()


Overwriting app.py


In [ ]:
from pyngrok import ngrok
# tunnels = ngrok.get_tunnels()
# tunnels

In [ ]:
ngrok.set_auth_token('2Y0P8SRzLjQLgBM9OD2jLJ1VVPE_6Y1hdbHbpAZZNqajcjsph')

In [ ]:
# Terminate open tunnels if exist
ngrok.kill()

# Set up a new tunnel
url = ngrok.connect(port=8501)

# Run the Streamlit app as a background process
!nohup streamlit run app.py &

# Print the public URL to access the Streamlit app
print('Streamlit URL:', url)


nohup: appending output to 'nohup.out'
Streamlit URL: http://28d8-35-240-131-1.ngrok-free.app


SIMILAR TEXT

In [ ]:
import nltk
import sklearn
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import string
nltk.download('stopwords')
nltk.download('punkt')
ps = PorterStemmer()

In [ ]:
def transform_text(text):
    text = text.lower() # lower case
    text = nltk.word_tokenize(text) # separating words

    # removing unnecessary words and punctuation
    y = []
    for i in text:
        if i.isalnum() and i not in stopwords.words('english') \
            and i not in string.punctuation:
            y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
# Import the necessary libraries
import pandas as pd

# Read the CSV file into a DataFrame
music_df = pd.read_csv('/content/music_mental_health.csv', header=0)

# Print the DataFrame to confirm that it was read correctly
print(music_df)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/music_mental_health.csv')

# 2. Removing duplicates
df = df.drop_duplicates()

# Save the cleaned DataFrame back to a CSV file
df.to_csv('/content/cleaned_music_mental_health.csv', index=False)


In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/music_mental_health.csv')

# 2. Removing duplicates
df = df.drop_duplicates()


# Save the cleaned DataFrame back to a CSV file
df.to_csv('/content/cleaned_music_mental_health.csv', index=False)
